In [1]:
from collections import namedtuple, defaultdict

In [5]:
import os

# Init

In [7]:
from nbdev.imports import *

In [8]:
create_config('nbdev-rewrite', 'flpeters')

In [9]:
if not os.environ.get("IN_TEST", None):
    assert IN_NOTEBOOK
    assert not IN_COLAB
    assert IN_IPYTHON

# Notebook Loading

In [10]:
#export
def read_nb(fname):
    "Read the notebook in `fname`."
    with open(Path(fname),'r', encoding='utf8') as f: return nbformat.reads(f.read(), as_version=4)

In [11]:
test_nb = read_nb('00_export.ipynb')

In [12]:
test_nb.keys()

dict_keys(['cells', 'metadata', 'nbformat', 'nbformat_minor'])

In [13]:
test_nb['metadata']

{'kernelspec': {'display_name': 'Python 3',
  'language': 'python',
  'name': 'python3'},
 'language_info': {'codemirror_mode': {'name': 'ipython', 'version': 3},
  'file_extension': '.py',
  'mimetype': 'text/x-python',
  'name': 'python',
  'nbconvert_exporter': 'python',
  'pygments_lexer': 'ipython3',
  'version': '3.7.3'},
 'toc': {'base_numbering': 1,
  'nav_menu': {},
  'number_sections': True,
  'sideBar': True,
  'skip_h1_title': False,
  'title_cell': 'Table of Contents',
  'title_sidebar': 'Contents',
  'toc_cell': False,
  'toc_position': {},
  'toc_section_display': True,
  'toc_window_display': False}}

In [14]:
f"{test_nb['nbformat']}.{test_nb['nbformat_minor']}"

'4.4'

In [15]:
test_nb['cells'][0]

{'cell_type': 'code',
 'execution_count': 73,
 'metadata': {},
 'outputs': [],
 'source': 'from collections import namedtuple, defaultdict'}

In [16]:
len(test_nb['cells'])

60

# Parsing Names

In [17]:
import ast
import _ast
from pprint import pprint

In [18]:
def print_tree(node):
    if isinstance(node, (list, tuple)):
        for x in node:
            print_tree(x)
    elif hasattr(node, '_fields'):
        for f in node._fields:
            # print(f)
            print_tree(node.__getattribute__(f))
    else:
        print(node)
        # pass

In [19]:
def run_tests(cases):
    tp = TestParser()
    for i, (n, c, r) in enumerate(cases):
        print(f'({i + 1} / {len(cases)}) TEST {n}:')
        try:
            tree = ast.parse(c)
            res = tp.visit(tree)
            if res == r:
                print(f'TEST RESULT: SUCCESS\n')
            else:
                raise Exception(f'TEST FAILED WITH RESULT: {res}\nEXPECTED: {r}')
        except Exception as e:
            print(f'TEST RESULT:\n{e}\n')
            # raise e
    print('--------------- ALL TESTS COMPLETED ---------------')

In [20]:
class TestParser(ast.NodeVisitor):
    def visit(self, node):
        """Visit a node."""
        method = 'visit_' + node.__class__.__name__
        visitor = getattr(self, method, self.generic_visit)
        print(f'{node.__class__.__name__} -> {visitor.__name__}')
        return visitor(node)
    
    def _default(self, node):
        # pprint(node.__dict__)
        print(f'attr:   {node._attributes}')
        print(f'fields: {node._fields}')
        print('-'*25)
        
    def visit_Assign(self, node):
        self._default(node)
    
    def visit_FunctionDef(self, node):
        self._default(node)
        for d in node.decorator_list:
            print(self.visit(d))
            
    def visit_ClassDef(self, node):
        self._default(node)

## Tests

In [21]:
test_assignment = [
('Default Assignment', """
a = 1
b = a
a = 2
""", None),
('Tuple unpacking', """
a, b = (1, 2)
""", None),
('unpacking to tuples and lists', """
(a, b) = (1, 2)
[a, b] = (1, 2)
""", None),
('Multiple assignments', """
a = b = 2
""", None),
('List Deconstruction', """
head, *tail = [1,2,3,4,5]
""", None),
('Private Variables', """
_a = 1
""", None),
]
run_tests(test_assignment)

(1 / 6) TEST Default Assignment:
Module -> generic_visit
Assign -> visit_Assign
attr:   ('lineno', 'col_offset')
fields: ('targets', 'value')
-------------------------
Assign -> visit_Assign
attr:   ('lineno', 'col_offset')
fields: ('targets', 'value')
-------------------------
Assign -> visit_Assign
attr:   ('lineno', 'col_offset')
fields: ('targets', 'value')
-------------------------
TEST RESULT: SUCCESS

(2 / 6) TEST Tuple unpacking:
Module -> generic_visit
Assign -> visit_Assign
attr:   ('lineno', 'col_offset')
fields: ('targets', 'value')
-------------------------
TEST RESULT: SUCCESS

(3 / 6) TEST unpacking to tuples and lists:
Module -> generic_visit
Assign -> visit_Assign
attr:   ('lineno', 'col_offset')
fields: ('targets', 'value')
-------------------------
Assign -> visit_Assign
attr:   ('lineno', 'col_offset')
fields: ('targets', 'value')
-------------------------
TEST RESULT: SUCCESS

(4 / 6) TEST Multiple assignments:
Module -> generic_visit
Assign -> visit_Assign
attr:  

In [22]:
test_funcdef = [
('Default function definition', """
def add(a, b):
    return a + b
""", None),
('Type Annotated function def', """
def calc(a:int, b:int) -> int:
    c:float = 2.0
    return (a + b) * c
""", None),
('function decorators', """
@test1
@test2
def add(a, b):
    return a + b
""", None),
('@patch and more complex type annotations', """
@patch
def func (obj:(Class1, Class2), a:int)->int:
    pass
""", None)
]
run_tests(test_funcdef)

(1 / 4) TEST Default function definition:
Module -> generic_visit
FunctionDef -> visit_FunctionDef
attr:   ('lineno', 'col_offset')
fields: ('name', 'args', 'body', 'decorator_list', 'returns')
-------------------------
TEST RESULT: SUCCESS

(2 / 4) TEST Type Annotated function def:
Module -> generic_visit
FunctionDef -> visit_FunctionDef
attr:   ('lineno', 'col_offset')
fields: ('name', 'args', 'body', 'decorator_list', 'returns')
-------------------------
TEST RESULT: SUCCESS

(3 / 4) TEST function decorators:
Module -> generic_visit
FunctionDef -> visit_FunctionDef
attr:   ('lineno', 'col_offset')
fields: ('name', 'args', 'body', 'decorator_list', 'returns')
-------------------------
Name -> generic_visit
Load -> generic_visit
None
Name -> generic_visit
Load -> generic_visit
None
TEST RESULT: SUCCESS

(4 / 4) TEST @patch and more complex type annotations:
Module -> generic_visit
FunctionDef -> visit_FunctionDef
attr:   ('lineno', 'col_offset')
fields: ('name', 'args', 'body', 'decor

In [23]:
test_classdef = [
('Default class definition', """
class Abc:
    pass
""", None),
('Default class def 2', """
class Abc():
    pass
""", None),
]
run_tests(test_classdef)

(1 / 2) TEST Default class definition:
Module -> generic_visit
ClassDef -> visit_ClassDef
attr:   ('lineno', 'col_offset')
fields: ('name', 'bases', 'keywords', 'body', 'decorator_list')
-------------------------
TEST RESULT: SUCCESS

(2 / 2) TEST Default class def 2:
Module -> generic_visit
ClassDef -> visit_ClassDef
attr:   ('lineno', 'col_offset')
fields: ('name', 'bases', 'keywords', 'body', 'decorator_list')
-------------------------
TEST RESULT: SUCCESS

--------------- ALL TESTS COMPLETED ---------------


# Markup Comments

In [24]:
import re

In [26]:
class KeywordParser:
    def __init__(self, *init_keywords):
        self.parsers = {}
        for kw in init_keywords: self.parsers[kw] = self._create_parser(kw)

    def _create_parser(self, keyword):
        pattern = fr"""
        ^              # start of line, since MULTILINE is passed
        \s*            # any amount of whitespace
        \#\s*          # literal "#", then any amount of whitespace
        {keyword}(.*)  # keyword followed by arbitrary symbols (except new line)
        $              # end of line, since MULTILINE is passed
        """
        return re.compile(pattern, re.IGNORECASE | re.MULTILINE | re.VERBOSE)
        
    def __getitem__(self, key):
        if key in self.parsers: return self.parsers[key]
        else:
            parser = self._create_parser(key)
            self.parsers[key] = parser
            return parser

In [27]:
OptionsTuple = namedtuple(typename='Options',
                          field_names=['export_target', 'internal'],
                          defaults=[None, False])

In [149]:
_re_legacy_options = re.compile(fr'^(i)?\s*(\S*)\s*$')

_re_legacy_options.search('   i').groups()

(None, 'i')

In [28]:
_re_legacy_options = re.compile(fr'^(i)?\s*(\S*)\s*$')
def legacy_parse_options(options:str) -> OptionsTuple:
    internal, export_target = _re_legacy_options.search(options).groups
    return OptionsTuple(export_target=export_target, internal=internal)

In [46]:
def parse_options(options:str, legacy:bool=True) -> OptionsTuple:
    if (options is None) or (options == '') or (options.isspace()): return OptionsTuple()
    else:
        if legacy: return legacy_parse_options(options)
        else:
            raise NotImplementedError('this branch of parse_options() is not implemented yet.')

In [30]:
def parse_export(source:str) -> (bool, OptionsTuple):
    keyword_parser = KeywordParser()
    res = keyword_parser['export'].search(source)
    if res: return (True, parse_options(res.groups()[0]))
    else: return (False, None)

In [43]:
def find_exports(cells:list, default:str, code_only:bool=True) -> list:
    # check for each cell if it's supposed to be exported and aggregate cell content together with export options
    # remove whitespace at end of lines
    exports = []
    for i, cell in enumerate(cells):
        if code_only and (cell.cell_type != 'code'): continue
        else:
            # print(i, cell.cell_type)
            source = cell.source
            # print(source)
            to_export, options = parse_export(source)
            if to_export:
                assert options.export_target or default, f'Cell nr.{i} doesn\'t have an export target, \
                                                           and a default is not specified:\n{source}'
                if not options.export_target: options = options._replace(export_target=default)
                assert options.export_target, f'something went wrong with export target: {options.export_target}'
                exports.append((source, options))
            else: continue
    return exports

# list((code, internal, export_target))

In [32]:
test_nb['cells'][0].keys()

dict_keys(['cell_type', 'execution_count', 'metadata', 'outputs', 'source'])

In [45]:
test_nb['cells'][0]

{'cell_type': 'code',
 'execution_count': 73,
 'metadata': {},
 'outputs': [],
 'source': 'from collections import namedtuple, defaultdict'}

In [44]:
find_exports(test_nb['cells'], 'export', code_only=True)

[('#export\ndef read_nb(fname):\n    "Read the notebook in `fname`."\n    with open(Path(fname),\'r\', encoding=\'utf8\') as f: return nbformat.reads(f.read(), as_version=4)',
  Options(export_target='export', internal=False)),
 ('test_markup = [\n(\'export\', """\n# export\n""", (1, 1, 1, 0)),\n(\'export internal\', """\n# export -i\n""", (1, 0, 0, 0)),\n(\'export show source\', """\n# export -s\n""", (1, 1, 1, 1)),\n(\'export internal show\', """\n# export -i -s\n""", (1, 0, 1, 1)),\n(\'default empty\', """\n\n""", None),\n(\'hide\', """\n# hide\n""", (0, 0, 0, 0)),\n(\'layout 1\', """\n#export\n""", (1, 1, 1, 0)),\n(\'layout 2\', """\n # export\n""", (1, 1, 1, 0)),\n(\'multiple comments\', """\n# export\n# hide\n""", None),\n(\'multi comment same line\', """\n# export hide\n""", None),\n(\'multiple comments default_exp\', """\n# export\n# default_exp\n""", None),\n]\nrun_markup_tests(test_markup)',
  Options(export_target='export', internal=False)),
 ('#export \ndef notebook2script(

## Tests

In [25]:
def run_markup_tests(cases):
    for i, (n, c, r) in enumerate(cases):
        print(f'({i + 1} / {len(cases)}) TEST {n}:')
        try:
            res = parse_markup(c)
            assert res == r, f'TEST FAILED WITH RESULT: {res}\nEXPECTED: {r}'
            print(f'TEST RESULT: SUCCESS\n')
        except Exception as e:
            print(f'TEST FAILED WITH EXCEPTION:\n{e}\n')
            # raise e
    print('--------------- ALL TESTS COMPLETED ---------------')

In [23]:
test_markup = [
('export', """
# export
""", (1, 1, 1, 0)),
('export internal', """
# export -i
""", (1, 0, 0, 0)),
('export show source', """
# export -s
""", (1, 1, 1, 1)),
('export internal show', """
# export -i -s
""", (1, 0, 1, 1)),
('default empty', """

""", None),
('hide', """
# hide
""", (0, 0, 0, 0)),
('layout 1', """
#export
""", (1, 1, 1, 0)),
('layout 2', """
 # export
""", (1, 1, 1, 0)),
('multiple comments', """
# export
# hide
""", None),
('multi comment same line', """
# export hide
""", None),
('multiple comments default_exp', """
# export
# default_exp
""", None),
]
run_markup_tests(test_markup)

(1 / 11) TEST export:
TEST FAILED WITH EXCEPTION:
TEST FAILED WITH RESULT: None
EXPECTED: (1, 1, 1, 0)

(2 / 11) TEST export internal:
TEST FAILED WITH EXCEPTION:
TEST FAILED WITH RESULT: None
EXPECTED: (1, 0, 0, 0)

(3 / 11) TEST export show source:
TEST FAILED WITH EXCEPTION:
TEST FAILED WITH RESULT: None
EXPECTED: (1, 1, 1, 1)

(4 / 11) TEST export internal show:
TEST FAILED WITH EXCEPTION:
TEST FAILED WITH RESULT: None
EXPECTED: (1, 0, 1, 1)

(5 / 11) TEST default empty:
TEST RESULT: SUCCESS

(6 / 11) TEST hide:
TEST FAILED WITH EXCEPTION:
TEST FAILED WITH RESULT: None
EXPECTED: (0, 0, 0, 0)

(7 / 11) TEST layout 1:
TEST FAILED WITH EXCEPTION:
TEST FAILED WITH RESULT: None
EXPECTED: (1, 1, 1, 0)

(8 / 11) TEST layout 2:
TEST FAILED WITH EXCEPTION:
TEST FAILED WITH RESULT: None
EXPECTED: (1, 1, 1, 0)

(9 / 11) TEST multiple comments:
TEST RESULT: SUCCESS

(10 / 11) TEST multi comment same line:
TEST RESULT: SUCCESS

(11 / 11) TEST multiple comments default_exp:
export | default_exp 

In [24]:
(0, 0, 0, 0), # -> # hide
(0, 0, 1, 0), # -> nothing (default)
(0, 0, 1, 1), # -> # show source
(1, 0, 0, 0), # -> # export internal
(1, 0, 1, 0), # -> # export internal show
(1, 0, 1, 1), # -> # export internal show source
(1, 1, 0, 0), # -> # export hide
(1, 1, 1, 0), # -> # export
(1, 1, 1, 1), # -> # export show source

((1, 1, 1, 1),)

# Export

In [35]:
class ExportCache:
    def __init__(self, default_export=None):
        self.tupletype = namedtuple(typename='exports', field_names=['export_code', 'export_names'])
        self.exports = defaultdict(self._create_exp)
        if default_export is not None: self[default_export]
    
    def _create_exp(self): return self.tupletype(export_code=list(), export_names=set())
    
    def __getitem__(self, key): return self.exports[key]
    
    def add_names(self, key, names):
        target = self[key].export_names
        for name in names: target.add(name)
            
    def add_code(self, key, code): self[key].export_code.append(code)

In [36]:
def find_default_export(cells:list) -> str:
    # search through all cells to find the default_exp keyword and return it's value.
    # syntax checking
    # maybe do some sanity checking
    return 'export'
    pass

In [37]:
def create_mod_file(orig_nbfname, targ_pyfname):
    # create the .py file in the correct folder, with a header saying where it was originally from
    pass

In [38]:
def find_names(code:str) -> list:
    # find function and variable names in this code block
    # find _all_ declarations
    pass

In [39]:
def _notebook2script(cells=None, fname=None, silent=False, to_dict=False):
    """Convert a single notebook"""
    if cells: print('cells is only used for testing purposes!')
    if fname is not None: raise NotImplementedError('fname is a "must pass", but not yet')
    # load notebook content
    # load config
    default = find_default_export(cells)
    if default is None:
        print('WARNING: No default export file found! (should this crash, or see if each export has its own target?)')
    else:
        # maybe this should be done at the bottom, together with all the others
        # create_mod_file(original_nbfile_path, target_pyfile_path) # flipped in original code
        pass
    export_cache = ExportCache(default)
    # load _nbdev file and create a spec from it (no idea why this is needed)
    exports = find_exports(cells, default)
    for j, (code, options)  in enumerate(exports):
        # code = clean_code(code)
        e, i = options.export_target, options.internal
        if not i: export_cache.add_names(e, find_names(code))
        export_cache.add_code(e, code)
    write_to_export_files(export_cache, default)
    # add names to _nbdev index
    # write code cell to file
    # save _nbdev file

In [40]:
#export 
def notebook2script(fname=None, silent=False, to_dict=False):
    "Convert notebooks matching `fname` to modules"
    # initial checks
    if os.environ.get('IN_TEST',0): return  # don't export if running tests
    if fname is None:
        reset_nbdev_module()
        update_version()
        update_baseurl()
        files = [f for f in Config().nbs_path.glob('*.ipynb') if not f.name.startswith('_')]
    else: files = glob.glob(fname)
    d = collections.defaultdict(list) if to_dict else None
    for f in sorted(files): d = _notebook2script(f, silent=silent, to_dict=d)
    if to_dict: return d
    else: add_init(Config().lib_path)